## Projekt Briefkastenfirmen

## Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile # für den Tor-Browser
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary # für den Tor-Browser
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
from stem import Signal # Um beim Tor-Surfer die IP/Idendity zu wechseln
from stem.control import Controller #Um beim Tor-Surfer die IP/Idendity zu wechseln
from selenium.webdriver.support.ui import WebDriverWait # Um die Seite ganz zu laden brauchte ich aber doch nicht.
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common import exceptions 
from selenium.common.exceptions import ElementNotInteractableException
from random import randint #für random time sleep
from time import sleep #für random time sleep 
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

# 1. Scrapen vom Handelsregister
- Ich will von allen 21000 Adressen des Kantons Zug die erste Page speichern. 
- Adressenaufbereitung: Siehe File **02 Adressen Aufbereitung**

## 1.1 Erste Seite aller Adressen abrufen und speichern
hier nun zuerst alle erste Seiten - so dass ich überall eine Anzahl habe. 
In einem weiteren Schritt könnte ich noch bei allen, die mehr als 20 Firmen haben (und damit mehr als eine Seite) die unterseiten durchgehen, um so an alle UIDs ranzukommen. Diese kann man dann via URL abrufen. Für meine erste Geschichte benötige ich diese aber nicht.
#### Dokumentation/Versuche unter 03 Scraping Handesregister/09 Tor-Scraping Handelsregister
- Die ersten rund 5000 konnte ich gut runterladen. Danach gingen immer weniger.
- Erst versuchte ich es mit 1000-er Päckchen nach welchen ich den Browser wieder neu öffnete. Ging nicht lange gut
- Dann die ersten Tor-Versuche: Versehentlich wechselte ich zuerst zu Firefox statt ganz zum Tor-Browser. Dies funktionierte besser als Chrome, aber auch nicht gut
- Dann fand ich den Dreh mit Tor heraus. Dies ging aber gar nicht mehr. 
- Danach die Lösung mit Chrome und den Exceptions. Auch hier konnte ich plötzlich nur noch 100-er Päckchen verarbeiten
-  Auch das Telefon mit der Handelsregister-IT brachte keine Lösung
- Finally: Mit Firefox arbeiten, ein Try/Except um das ganze und nach jedem Error das Fenster schiessen, Cookies löschen (wohl der wichtigste Teil der Sache) und wieder neu öffnen. So konnte ich die fehlenden rund 10'000 Files doch noch scrapen. 

#### Abflauf Scraping:
- Zuerst zog ich die ganze Stadt Zug runter. Notfalls hätte man es auch damit machen können. Denn gemäss dem Leiter des Handesregisteramtes sind die relevanten Adressen wohl vorallem in der Stadt.

### Ein Frame für die Stadt und eines für Gemeinden
Ich mache zwei separate Frames. Jenes der Stadt wurde mit demselben Code erstellt, jedoch bereits früher gemacht. Hier nun noch die Gemeinden

In [6]:
dff=pd.read_csv(folderpath+"/BRFrames/alle_Adressen_ohne_Stadt") #alle_Adressen: siehe 02 Adressen Aufbereitung

In [ ]:
counter=1000
nummer=21




#Wir starten den Browser auf
driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # hier mit dem Firefox (weil zuerst gedacht das sei schon der Tor)
binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
time.sleep(randint(3,7)) #random time.sleep. In der Hoffnung dass er mich so weniger erkennt
#Hier drücke ich den Cookie-Banner weg
cookie=driver.find_element_by_id("j_idt198")
time.sleep(randint(3,7))            
cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
sleep(randint(3,7))
# Hier klappe ich die Zusätzlichen Suchfelder auf
driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
time.sleep(3)

for adress in tqdm(dff["Adresse"]): #die Adressen durchgehen
        # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
    try:
        su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
        time.sleep(randint(1,3))
        driver.execute_script('arguments[0].click();', su_fel_ad)
        time.sleep(randint(1,2))
        filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
        time.sleep(randint(1,3))
        filt.send_keys('Domiziladresse')
        filt.send_keys(Keys.ENTER)
        time.sleep(randint(1,4))

        search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
        search_adress.send_keys(adress) #Adresse Eingeben
        search_adress.send_keys(Keys.ENTER) #Enter drücken
        time.sleep(randint(5,10))
        page = driver.page_source.encode("utf-8") #Page in Variable speichern

        with open (folderpath+"/HRSeiten_Gemeinden/frontpage"+str(nummer)+"_"+str(counter)+".htm", "wb+") as file:
            file.write(page)
            file.close()

        pagename = "frontpage"+str(nummer)+"_"+str(counter)+".htm"        
        counter=counter+1 

         #Hier eine Try-Schlaufe weil auch das Clear nicht immer funktioniert. hat sich bewährt
        clear_all=driver.find_element_by_id("idSucheForm:j_idt167").click()
        time.sleep(randint(2,7))

    except StaleElementReferenceException:      
        print("musste neu starten - Stale")
         
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # hier mit dem Firefox (weil zuerst gedacht das sei schon der Tor)
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
        time.sleep(randint(3,7)) #random time.sleep. In der Hoffnung dass er mich so weniger erkennt
        #Hier drücke ich den Cookie-Banner weg
        cookie=driver.find_element_by_id("j_idt198")
        time.sleep(randint(3,7))            
        cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
        sleep(randint(3,7))
        # Hier klappe ich die Zusätzlichen Suchfelder auf
        driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
        time.sleep(3)
        

    except ElementNotInteractableException:
        print("musste neu starten - Element not")
       # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
         # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # hier mit dem Firefox (weil zuerst gedacht das sei schon der Tor)
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
        time.sleep(randint(3,7)) #random time.sleep. In der Hoffnung dass er mich so weniger erkennt
        #Hier drücke ich den Cookie-Banner weg
        cookie=driver.find_element_by_id("j_idt198")
        time.sleep(randint(3,7))            
        cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
        sleep(randint(3,7))
        # Hier klappe ich die Zusätzlichen Suchfelder auf
        driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
        time.sleep(3)
        
   
   

# 2. Nun Infos roh rausziehen
Roheinträge aus den gespeicherten Files abfragen
Danach mit Split in Pandas Infos rausziehen (Herleitung siehe "04 Auslesen der Anzahl Firmen")

In [2]:
anzahl_liste=[]
path=folderpath+"/HRSeiten_Gemeinden/"
for page in tqdm(os.listdir(path)): #Ich erstelle direkt aus dem Folder eine Liste.
    file=open(path+page, 'r')
    page=page
    text=file.read()
    soup=BeautifulSoup(text, 'html.parser')
    div_list=soup.find_all("div")
    if len(div_list) == 97: #Wenn es keine Firma hat, hat es nur 97 Listen-Punkte
        anzahl= str("0") # Ich vermerke, dass es keine Firmen hat
        input_list=div_list[50].find_all("input") # Hier der Adressen-Eintrag
        adresse_o=input_list[0]
        adresse=str(adresse_o) #HIer noch ein Trick, dass er mir die Sache nicht als Liste ausspuckt
       
    else:
        anzahl= div_list[-26].text #Angabe mit Anzahl Firmen auslesen.
        input_list_f=div_list[-51].find_all("input") 
        adresse_f=input_list_f[0]
        adresse=str(adresse_f) # hier derselbe Trick wie oben
        

    minidict_a={"Seite":page,"Anzahl":anzahl, "Adresse":adresse}
    anzahl_liste.append(minidict_a)

100%|██████████| 24526/24526 [23:56<00:00, 17.07it/s]


In [3]:
# Hier nun die rohe Liste mit den Adressen, Files und Anzahl Firmen.
df_adan=pd.DataFrame(anzahl_liste)
df_adan.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_direkt_ab_ausleser")
df_adan

,Seite,Anzahl,Adresse
0,frontpage0.htm,\n\t\t\t\tNo. of companies found: 1\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
1,frontpage0_0.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
2,frontpage0_1.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
3,frontpage0_10.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
4,frontpage0_100.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
...,...,...,...
24521,frontpage9_2035.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
24522,frontpage9_2036.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
24523,frontpage9_2037.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
24524,frontpage9_2038.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."


In [21]:
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_direkt_ab_ausleser")

In [22]:
df_adan.drop_duplicates(subset=["Anzahl", "Adresse"],inplace=True) #alle die hier doppelt sind fallen weg.

In [23]:
df_adan.shape

(15990, 4)

In [24]:
df_adan.drop_duplicates(subset=["Adresse"],inplace=True) 

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [15]:
df_adan.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_ausgelesen_roh")# abspeichern

In [18]:
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_ausgelesen_roh")
df_adan

,Unnamed: 0,Unnamed: 0.1,Seite,Anzahl,Adresse
0,0,0,frontpage0.htm,\n\t\t\t\tNo. of companies found: 1\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
1,1,1,frontpage0_0.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
2,2,2,frontpage0_1.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
3,3,3,frontpage0_10.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
4,4,4,frontpage0_100.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
...,...,...,...,...,...
15985,24490,24490,frontpage9_2004.htm,\n\t\t\t\tNo. of companies found: 1\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
15986,24491,24491,frontpage9_2005.htm,\n\t\t\t\tNo. of companies found: 1\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
15987,24493,24493,frontpage9_2007.htm,\n\t\t\t\tNo. of companies found: 4\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
15988,24495,24495,frontpage9_2009.htm,\n\t\t\t\tNo. of companies found: 2\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."


In [26]:
df_adan.shape, dff.shape #25 zu viel - hmm... bei dieser Menge wohl ok

((15687, 4), (15662, 15))

## 2.1 Zahl und Adresse extrahieren
mit Regex vom Rest trennen und damit Zahlen und Adresse aus den Daten extrahieren.. 

In [ ]:
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_ausgelesen_roh")

In [27]:
# Hier nun die Formel, um aus allen die Zahl rausziehen zu können
df_adan["Anzahl"]=df_adan["Anzahl"].str.extract("(\d+)")

In [28]:
# und noch die Adresse
df_adan["Adresse"]=df_adan["Adresse"].str.extract("value=\"(.*)\"")

In [29]:
df_adan

,Unnamed: 0,Seite,Anzahl,Adresse
0,0,frontpage0.htm,1,"Marktgasse 12, Baar"
1,1,frontpage0_0.htm,NaN,NaN
2,2,frontpage0_1.htm,0,"Langgasse 4, Baar"
3,3,frontpage0_10.htm,0,"Oberdorfstrasse 1, Baar"
4,4,frontpage0_100.htm,0,"Grundstrasse 12, Baar"
...,...,...,...,...
24423,24423,frontpage9_1937.htm,0,"Gartenstrasse 1b, HÃ¼nenberg"
24424,24424,frontpage9_1938.htm,0,"Gartenstrasse 1c, HÃ¼nenberg"
24425,24425,frontpage9_1939.htm,0,"Gartenstrasse 1c, HÃ¼nenberg"
24427,24427,frontpage9_1941.htm,0,"Chamerstrasse 23, HÃ¼nenberg"


In [30]:
df_adan[df_adan["Adresse"].notnull()==True] # Nun will ich nur jene, die keine NANs haben

,Unnamed: 0,Seite,Anzahl,Adresse
0,0,frontpage0.htm,1,"Marktgasse 12, Baar"
2,2,frontpage0_1.htm,0,"Langgasse 4, Baar"
3,3,frontpage0_10.htm,0,"Oberdorfstrasse 1, Baar"
4,4,frontpage0_100.htm,0,"Grundstrasse 12, Baar"
5,5,frontpage0_101.htm,0,"Grundstrasse 14, Baar"
...,...,...,...,...
24423,24423,frontpage9_1937.htm,0,"Gartenstrasse 1b, HÃ¼nenberg"
24424,24424,frontpage9_1938.htm,0,"Gartenstrasse 1c, HÃ¼nenberg"
24425,24425,frontpage9_1939.htm,0,"Gartenstrasse 1c, HÃ¼nenberg"
24427,24427,frontpage9_1941.htm,0,"Chamerstrasse 23, HÃ¼nenberg"


In [31]:
df_adan.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_HR_extrahiert_unbereinigt")

## 2.2 Umlaut-Codes bereinigen
wichtig für den Abgleich mit der Adress-Liste. (Herleitung siehe: 06 Bereinigung Umlaute)
- So sehe ich, ob Adressen noch fehlen
- Kann so später die Dataframes wieder zusammenfügen für Geopandas

In [36]:
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_HR_extrahiert_unbereinigt")
df_adan

,Unnamed: 0,Unnamed: 0.1,Seite,Anzahl,Adresse
0,0,0,frontpage0.htm,1.0,"Marktgasse 12, Baar"
1,1,1,frontpage0_0.htm,NaN,NaN
2,2,2,frontpage0_1.htm,0.0,"Langgasse 4, Baar"
3,3,3,frontpage0_10.htm,0.0,"Oberdorfstrasse 1, Baar"
4,4,4,frontpage0_100.htm,0.0,"Grundstrasse 12, Baar"
...,...,...,...,...,...
15682,24423,24423,frontpage9_1937.htm,0.0,"Gartenstrasse 1b, HÃ¼nenberg"
15683,24424,24424,frontpage9_1938.htm,0.0,"Gartenstrasse 1c, HÃ¼nenberg"
15684,24425,24425,frontpage9_1939.htm,0.0,"Gartenstrasse 1c, HÃ¼nenberg"
15685,24427,24427,frontpage9_1941.htm,0.0,"Chamerstrasse 23, HÃ¼nenberg"


In [37]:
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¼","ü")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¶","ö")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¤","ä")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã„","Ä")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã–","Ö")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ãœ","Ü")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã©","é")

In [38]:
# ich entferne die, die gleich sind und sowohl Adresse wie auch Anzahl enthalten
df_adan.drop_duplicates(subset=["Adresse"], inplace=True)

In [39]:
df_adan.shape 

(15653, 5)

In [40]:
df_adan.to_csv(folderpath+"/BRFrames/alle_Gemeinden_HR_extrahiert_umlaute_weg")

# 3. Abgleichen von Adress-Frame und HR-Scrape-Frame
Sind alle Adressen runtergezogen? 
- Ich merge das Adress-Frame mit dem HR-Frame an der Adresse und  "outer" und schaue dann, ob es noch irgendwo ein NAN hat.

In [ ]:
df_adan=pd.read_csv(folderpath+"/BRFrames/alle_Gemeinden_HR_extrahiert_umlaute_weg")

In [11]:
df_adr=pd.read_csv(folderpath+"/BRFrames/Alle_Adressen_ohne_Stadt") # die ursprünglichen Adressen 

In [12]:
df_adr.shape

(15662, 15)

In [13]:
# so zusammenführen dass auch die fehlenden Daten sichtbar werden
df_abgleich=pd.merge(df_adan, df_adr, on="Adresse", how="outer") 

In [14]:
df_abgleich.drop_duplicates(subset=["Adresse", "Anzahl"], inplace=True)# Doppelte Einträge löschen

In [15]:
df_abgleich

,Seite,Anzahl,Adresse,Unnamed: 0,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,frontpage0.htm,1,"Marktgasse 12, Baar",0.0,310003.0,0.0,ZG,1701.0,Baar,Marktgasse,12,6340.0,0.0,Baar,2682511.588,1227773.739,de
2,frontpage0_0.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,frontpage12_3693.htm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,frontpage0_1.htm,0,"Langgasse 4, Baar",1008.0,311170.0,0.0,ZG,1701.0,Baar,Langgasse,4,6340.0,0.0,Baar,2682635.135,1227846.756,de
7,frontpage0_10.htm,0,"Oberdorfstrasse 1, Baar",1017.0,311181.0,0.0,ZG,1701.0,Baar,Oberdorfstrasse,1,6340.0,0.0,Baar,2682625.433,1227807.127,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15997,NaN,NaN,"Obere Rainstrasse 4a, Neuheim",277.0,3155638.0,0.0,ZG,1705.0,Neuheim,Obere Rainstrasse,4a,6345.0,0.0,Neuheim,2686175.994,1228711.477,de
15998,NaN,NaN,"Schneitstrasse 13, Oberägeri",238.0,315613.0,0.0,ZG,1706.0,Oberägeri,Schneitstrasse,13,6315.0,0.0,Oberägeri,2688690.243,1221460.363,de
15999,NaN,NaN,"Hauptseestrasse 81, Morgarten",583.0,316017.0,0.0,ZG,1706.0,Oberägeri,Hauptseestrasse,81,6315.0,3.0,Morgarten,2691386.636,1218630.484,de
16000,NaN,NaN,"Berchtwilerstrasse 1, Rotkreuz",260.0,316538.0,0.0,ZG,1707.0,Risch,Berchtwilerstrasse,1,6343.0,0.0,Rotkreuz,2674958.501,1221777.314,de


In [16]:
df_abgleich.dropna(subset=['GDENAME'],inplace=True) #Hier noch jene löschen, die keine Infos haben. 

In [17]:
df_abgleich[df_abgleich["Anzahl"].notnull()==False] # Hier jene, die ich von Hand prüfte (nach x-mal scrapen)

,Seite,Anzahl,Adresse,Unnamed: 0,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
15990,NaN,NaN,"Lorzenweidstrasse 109, Hagendorn",1083.0,313164.0,0.0,ZG,1702.0,Cham,Lorzenweidstrasse,109,6332.0,0.0,Hagendorn,2675275.989,1228448.801,de
15991,NaN,NaN,"Lindenbergstrasse 6b, Hünenberg",253.0,313636.0,0.0,ZG,1703.0,Hünenberg,Lindenbergstrasse,6b,6331.0,0.0,Hünenberg,2674945.743,1225881.991,de
15992,NaN,NaN,"Sonnhaldenstrasse 50a, Hünenberg",464.0,313855.0,0.0,ZG,1703.0,Hünenberg,Sonnhaldenstrasse,50a,6331.0,0.0,Hünenberg,2674585.247,1225922.150,de
15993,NaN,NaN,"Luzernerstrasse 86, Hünenberg See",717.0,314177.0,0.0,ZG,1703.0,Hünenberg,Luzernerstrasse,86,6333.0,0.0,Hünenberg See,2676700.412,1225388.057,de
15994,NaN,NaN,"Oberer Chämletenweg 31b, Hünenberg See",928.0,3120334.0,0.0,ZG,1703.0,Hünenberg,Oberer Chämletenweg,31b,6333.0,0.0,Hünenberg See,2676496.431,1224993.155,de
15995,NaN,NaN,"Kembergstrasse 39, Hünenberg See",1205.0,160004790.0,0.0,ZG,1703.0,Hünenberg,Kembergstrasse,39,6333.0,0.0,Hünenberg See,2676212.573,1225291.354,de
15996,NaN,NaN,"Edlibachstrasse 15, Neuheim",201.0,315237.0,0.0,ZG,1705.0,Neuheim,Edlibachstrasse,15,6345.0,0.0,Neuheim,2686042.318,1228201.772,de
15997,NaN,NaN,"Obere Rainstrasse 4a, Neuheim",277.0,3155638.0,0.0,ZG,1705.0,Neuheim,Obere Rainstrasse,4a,6345.0,0.0,Neuheim,2686175.994,1228711.477,de
15998,NaN,NaN,"Schneitstrasse 13, Oberägeri",238.0,315613.0,0.0,ZG,1706.0,Oberägeri,Schneitstrasse,13,6315.0,0.0,Oberägeri,2688690.243,1221460.363,de
15999,NaN,NaN,"Hauptseestrasse 81, Morgarten",583.0,316017.0,0.0,ZG,1706.0,Oberägeri,Hauptseestrasse,81,6315.0,3.0,Morgarten,2691386.636,1218630.484,de


In [18]:
df_abgleich.shape, df_adr.shape #Kontrollieren ob es gleich viele sind wie bei der Stadt Zug

((15705, 17), (15662, 15))

## 3.1 Wenn die Länge noch nicht stimmt und es noch "NaN"-Werte hat

#### Alle Adressen filtern, die NAN-Werte haben - also nicht funktionierten https://stackoverflow.com/questions/22551403/python-pandas-filtering-out-nan-from-a-data-selection-of-a-column-of-strings

In [ ]:
df_fehlende=df_abgleich[df_abgleich["Anzahl"].notnull()==False] 

In [ ]:
df_fehlende # es hat unter diesen 12 Einträgen nur bei zweien je eine Firma. (von Hand geprüft)
#Ich lasse es nun sein

In [ ]:
df_fehlende.to_csv(folderpath+"/BRFrames/alle_Gemeinden_fehlende_adressem")

In [19]:
df_sauber=df_abgleich[df_abgleich["Anzahl"].notnull()==True]
df_sauber

,Seite,Anzahl,Adresse,Unnamed: 0,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,frontpage0.htm,1,"Marktgasse 12, Baar",0.0,310003.0,0.0,ZG,1701.0,Baar,Marktgasse,12,6340.0,0.0,Baar,2682511.588,1227773.739,de
6,frontpage0_1.htm,0,"Langgasse 4, Baar",1008.0,311170.0,0.0,ZG,1701.0,Baar,Langgasse,4,6340.0,0.0,Baar,2682635.135,1227846.756,de
7,frontpage0_10.htm,0,"Oberdorfstrasse 1, Baar",1017.0,311181.0,0.0,ZG,1701.0,Baar,Oberdorfstrasse,1,6340.0,0.0,Baar,2682625.433,1227807.127,de
8,frontpage0_100.htm,0,"Grundstrasse 12, Baar",100.0,310142.0,0.0,ZG,1701.0,Baar,Grundstrasse,12,6340.0,0.0,Baar,2682540.448,1227394.073,de
9,frontpage0_101.htm,0,"Grundstrasse 14, Baar",101.0,310144.0,0.0,ZG,1701.0,Baar,Grundstrasse,14,6340.0,0.0,Baar,2682546.158,1227373.838,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15984,frontpage9_1936.htm,2,"Gartenstrasse 1a, Hünenberg",260.0,313643.0,0.0,ZG,1703.0,Hünenberg,Gartenstrasse,1a,6331.0,0.0,Hünenberg,2674934.962,1225784.123,de
15985,frontpage9_1937.htm,0,"Gartenstrasse 1b, Hünenberg",261.0,313644.0,0.0,ZG,1703.0,Hünenberg,Gartenstrasse,1b,6331.0,0.0,Hünenberg,2674925.243,1225796.919,de
15986,frontpage9_1938.htm,0,"Gartenstrasse 1c, Hünenberg",262.0,313645.0,0.0,ZG,1703.0,Hünenberg,Gartenstrasse,1c,6331.0,0.0,Hünenberg,2674919.238,1225810.498,de
15988,frontpage9_1941.htm,0,"Chamerstrasse 23, Hünenberg",265.0,313649.0,0.0,ZG,1703.0,Hünenberg,Chamerstrasse,23,6331.0,0.0,Hünenberg,2675022.878,1225741.743,de


## 3.2 Wenns stimmt, alle Adressen da sind und die Umlaute bereinigt sind:
### Abspeichern
Die letzten 15 fehlenden habe ich von Hand kontrolliert. Es hat nur 2 Adressen mit je einer Firma darunter. Ich lasse es sein.


In [ ]:
df_sauber.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_Adressen_HR_sauber")

## 3.3 Dataframe Stadt und Gemeinden nun wieder zusammenbringen

In [41]:
df_gem=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_Adressen_HR_sauber")
df_gem

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0.1,Seite,Anzahl,Adresse,Unnamed: 0_y,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,0,0.0,0.0,frontpage0.htm,1.0,"Marktgasse 12, Baar",0.0,310003.0,0.0,ZG,1701.0,Baar,Marktgasse,12,6340.0,0.0,Baar,2682511.588,1227773.739,de
1,3,2.0,2.0,frontpage0_1.htm,0.0,"Langgasse 4, Baar",1008.0,311170.0,0.0,ZG,1701.0,Baar,Langgasse,4,6340.0,0.0,Baar,2682635.135,1227846.756,de
2,4,3.0,3.0,frontpage0_10.htm,0.0,"Oberdorfstrasse 1, Baar",1017.0,311181.0,0.0,ZG,1701.0,Baar,Oberdorfstrasse,1,6340.0,0.0,Baar,2682625.433,1227807.127,de
3,5,4.0,4.0,frontpage0_100.htm,0.0,"Grundstrasse 12, Baar",100.0,310142.0,0.0,ZG,1701.0,Baar,Grundstrasse,12,6340.0,0.0,Baar,2682540.448,1227394.073,de
4,6,5.0,5.0,frontpage0_101.htm,0.0,"Grundstrasse 14, Baar",101.0,310144.0,0.0,ZG,1701.0,Baar,Grundstrasse,14,6340.0,0.0,Baar,2682546.158,1227373.838,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15688,15692,24422.0,24422.0,frontpage9_1936.htm,2.0,"Gartenstrasse 1a, Hünenberg",260.0,313643.0,0.0,ZG,1703.0,Hünenberg,Gartenstrasse,1a,6331.0,0.0,Hünenberg,2674934.962,1225784.123,de
15689,15693,24423.0,24423.0,frontpage9_1937.htm,0.0,"Gartenstrasse 1b, Hünenberg",261.0,313644.0,0.0,ZG,1703.0,Hünenberg,Gartenstrasse,1b,6331.0,0.0,Hünenberg,2674925.243,1225796.919,de
15690,15694,24424.0,24424.0,frontpage9_1938.htm,0.0,"Gartenstrasse 1c, Hünenberg",262.0,313645.0,0.0,ZG,1703.0,Hünenberg,Gartenstrasse,1c,6331.0,0.0,Hünenberg,2674919.238,1225810.498,de
15691,15695,24427.0,24427.0,frontpage9_1941.htm,0.0,"Chamerstrasse 23, Hünenberg",265.0,313649.0,0.0,ZG,1703.0,Hünenberg,Chamerstrasse,23,6331.0,0.0,Hünenberg,2675022.878,1225741.743,de


In [42]:
df_sta=pd.read_csv(folderpath+"/BRFrames/Stadt_Zug_sauber")
df_sta

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0.1,Seite,Anzahl,Adresse,Unnamed: 0_y,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,0,0,0,frontpage0.htm,0,"Aabachstrasse 1, Zug",3464,190988543,0,ZG,1711,Zug,Aabachstrasse,1,6300,0,Zug,2681243.278,1225319.466,de
1,1,1,1,frontpage1.htm,5,"Aabachstrasse 10, Zug",3369,190564448,0,ZG,1711,Zug,Aabachstrasse,10,6300,0,Zug,2681328.366,1225274.400,de
2,2,2,2,frontpage10.htm,0,"Aabachstrasse 2, Zug",1430,321148,0,ZG,1711,Zug,Aabachstrasse,2,6300,0,Zug,2681294.530,1225154.193,de
3,3,3,3,frontpage100.htm,0,"Alte Baarerstrasse 4, Zug",1203,320895,0,ZG,1711,Zug,Alte Baarerstrasse,4,6300,0,Zug,2682278.228,1225122.918,de
4,4,4,4,frontpage1000.htm,0,"Chamerstrasse 180.2, Zug",4077,191775911,0,ZG,1711,Zug,Chamerstrasse,180.2,6300,0,Zug,2678903.909,1226520.613,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,4685,5645,5645,frontpage995.htm,18,"Chamerstrasse 175, Zug",2928,160009603,0,ZG,1711,Zug,Chamerstrasse,175,6300,0,Zug,2679236.637,1226168.313,de
4686,4686,5646,5646,frontpage996.htm,90,"Chamerstrasse 176, Zug",2957,160012752,0,ZG,1711,Zug,Chamerstrasse,176,6300,0,Zug,2679346.948,1226153.547,de
4687,4687,5647,5647,frontpage997.htm,0,"Chamerstrasse 177, Zug",2919,160007699,0,ZG,1711,Zug,Chamerstrasse,177,6300,0,Zug,2679221.076,1226153.141,de
4688,4688,5648,5648,frontpage998.htm,10,"Chamerstrasse 18, Zug",1433,321151,0,ZG,1711,Zug,Chamerstrasse,18,6300,0,Zug,2681295.335,1225130.509,de


In [43]:
#bestens, sehen beide gleich aus. Da können wir concat machen. bestens!
#Alle Adressen sind 20352. Offenbar sind noch welche doppelt oder so
df_all=pd.concat([df_sta, df_gem])
df_all.shape  

(20383, 20)

# 4. Scraping Telefonbuch
Nach dem ersten Visieren der Daten sowie dem Auffliegen der Luanda Leaks mit einem Akteur in Zug, wir mir bewusst, dass ich etwas benötige, mit dem ich die Grösse eines Gebäudes abschätzen kann. Leider ergaben die Recherchen bei Post sowie dem Kanton und Bund keine bessere Lösung als das Telefonbuch. Registriert werden nämlich nur Wohnungen. Geäude ohne Wohnungen sind gar nicht im Register. Das Telefonbuch soll also als Hilfe dienen, auch kleinere Hubs - wie das Luanda Leak, dass zu einem Haus in der Altstadt führt, in dem 40! Firmen registriert sind (normalerweise max. 3 Parteien pro Haus)- ausfindig zu machen. 

### Vorgehen
- Ich filtere all jene Adressen raus, bei denen überhaupt Firmen registriert sind.
- Diese werden dann beim Telefonbuch abgefragt. (So müssen nicht alle 21000 Adressen durchsucht werden)
- hier verwende ich einen gleichen Scraper wie beim HR. Auch mit Firefox und Cookies löschen. 
- Die Adressen der Stadt Zug habe ich in einem ersten Gang bereits hier hindurch gelassen

In [ ]:
df_af=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_Adressen_HR_sauber")

In [ ]:
df_af["Anzahl"]=df_af["Anzahl"].astype("int")

In [ ]:
# Alle Adressen mit mindestens einem Eintrag
df_af_s=df_af[df_af["Anzahl"]>0]

In [ ]:
counter=1

driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') 
binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get("https://tel.search.ch/")
for adresse in tqdm(df_af_s["Adresse"]):
    try:
        eingabe=driver.find_element_by_name("wo")
        eingabe.send_keys(adresse) #Adresse Eingeben
        time.sleep(randint(2,4))
        eingabe.send_keys(Keys.ENTER) #Enter drücken
        time.sleep(randint(5,6))
        telpage = driver.page_source.encode("utf-8") #Page in Variable speichern

        with open (folderpath+"/Telbuch_Seiten_Gemeinden/telfront"+str(counter)+".htm", "wb+") as file:
            file.write(telpage)
            file.close()

         
        counter=counter+1 

        clear_wo=driver.find_element_by_name("wo")
        clear_wo.clear()
        time.sleep(randint(2,4))
    
    except StaleElementReferenceException:      
        print("musste neu starten - Stale")
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # jetzt via Tor mit unterschiedlichen IP's
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get("https://tel.search.ch/")

    except ElementNotInteractableException:
        print("musste neu starten - Element not")
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # jetzt via Tor mit unterschiedlichen IP's
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get("https://tel.search.ch/")
        
    except NoSuchElementException:
        print("musste neu starten - No such element")
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # jetzt via Tor mit unterschiedlichen IP's
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get("https://tel.search.ch/")
        




## 4.1 Auslesen der Seiten
Wie schon beim Handesregister nehme ich auch hier den Teil der Adresse als ganzes raus und extrahiere danach mit demselben Code die Adresse. Auch die Umlautbereinigung geht gleich.
Die Stadt habe ich bereits erledigt(csv:"Telbuch_Stadt_Adressen_extrahiert_mit_richtigen_Umlauten"). Hier nun die Gemeinden

In [ ]:
anzahl_liste_t=[]
path_t=folderpath+"/Telbuch_Seiten_Gemeinden/"
for page_t in os.listdir(path_t):
    file_t=open(path_t+page_t,encoding="mbcs")
    text_t=file_t.read()
    soup_t=BeautifulSoup(text_t, 'html.parser')
    div_t_list=soup_t.find_all("div")
    anzahl_t=div_t_list[11].find_all("span")[0].text
    input_list=div_t_list[12].find_all("input")
    adresse_input=input_list[1]
    adresse_t=str(adresse_input)

    minidict_t={"Seite":page_t,"Anzahl":anzahl_t, "Adresse":adresse_t}
    anzahl_liste_t.append(minidict_t)

In [ ]:
dftel_t=pd.DataFrame(anzahl_liste_t) #in ein Dataframe
dftel_t.to_csv(folderpath+"\Telbook_alle_Gemeinden_roh")# und speichern
dftel_t

In [45]:
dftel_t=pd.read_csv(folderpath+"\Telbook_alle_Gemeinden_roh")

In [46]:
dftel_t["Adresse"]=dftel_t["Adresse"].str.extract("value=\"(.*)\"") #Adresse auslesen mit demselben Code wie HR

In [47]:
dftel_t.to_csv(folderpath+"/BRFrames/Telbuch_alle_Gemeinden_extrahiert")

In [48]:
dftel_t=pd.read_csv(folderpath+"/BRFrames/Telbuch_alle_Gemeinden_extrahiert")

In [49]:
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã¼","ü")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã¶","ö")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã¤","ä")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã„","Ä")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã–","Ö")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ãœ","Ü")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã©","é")

In [ ]:
dftel_t.to_csv(folderpath+"/BRFrames/Telbuch_alle_Gemeinden_Adressen_extrahiert_mit_richtigen_Umlauten")

# Handelsregister und Telbuch zusammenbringen
An der Adresse mergen. mal mit outer, damit ich sehe ob ich "überschuss" oder sonstige Ungereimtheiten habe.
Das Ergebnis für die Stadt: csv:"Stadt_sauber_HR_und_Telbuch"

In [50]:
dftel_t=pd.read_csv(folderpath+"/BRFrames/Telbuch_alle_Gemeinden_Adressen_extrahiert_mit_richtigen_Umlauten")

In [51]:
df_sauber=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_Adressen_HR_sauber")

In [52]:
df_ht=pd.merge(dftel_t, df_sauber, how="outer", on="Adresse")

In [53]:
df_ht1=df_ht[df_ht["Anzahl_y"]>0] #Alle einträge die mindestens eine Firma haben. Es hat noch Nans.
df_ht1

,Unnamed: 0_x,Seite_x,Anzahl_x,Adresse,Unnamed: 0_y,Unnamed: 0_x,Unnamed: 0.1,Seite_y,Anzahl_y,Unnamed: 0_y,...,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,0.0,telfront10000.htm,12,"Zugerstrasse 74, Baar",11761.0,16349.0,16349.0,frontpage2_976.htm,164.0,2034.0,...,1701.0,Baar,Zugerstrasse,74,6340.0,0.0,Baar,2681933.977,1226416.568,de
1,1.0,telfront10001.htm,57,"Mühlegasse 18, Baar",11953.0,17938.0,17938.0,frontpage30_262.htm,92.0,2076.0,...,1701.0,Baar,Mühlegasse,18,6340.0,0.0,Baar,2682425.299,1228159.836,de
3,2.0,telfront10002.htm,13,"Blegistrasse 11b, Baar",11717.0,16305.0,16305.0,frontpage2_936.htm,88.0,1994.0,...,1701.0,Baar,Blegistrasse,11b,6340.0,0.0,Baar,2685739.840,1230044.629,de
4,3.0,telfront10003.htm,39,"Sihlbruggstrasse 105, Baar",11270.0,15855.0,15855.0,frontpage2_530.htm,80.0,1559.0,...,1701.0,Baar,Sihlbruggstrasse,105,6340.0,0.0,Baar,2684903.579,1229684.742,de
5,4.0,telfront10004.htm,16,"Ruessenstrasse 12, Baar",11747.0,16335.0,16335.0,frontpage2_963.htm,75.0,2021.0,...,1701.0,Baar,Ruessenstrasse,12,6340.0,0.0,Baar,2685107.750,1229580.996,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13718,NaN,NaN,NaN,"Hauptstrasse 28, Oberägeri",12345.0,18533.0,18533.0,frontpage30_3155.htm,1.0,82.0,...,1706.0,Oberägeri,Hauptstrasse,28,6315.0,0.0,Oberägeri,2688917.988,1221178.499,de
13723,NaN,NaN,NaN,"Früebergstrasse 47, Baar",12350.0,18538.0,18538.0,frontpage30_316.htm,1.0,2443.0,...,1701.0,Baar,Früebergstrasse,47,6340.0,0.0,Baar,2682162.578,1228387.387,de
13725,NaN,NaN,NaN,"Hauptstrasse 40, Oberägeri",12352.0,18539.0,18539.0,frontpage30_3160.htm,2.0,87.0,...,1706.0,Oberägeri,Hauptstrasse,40,6315.0,0.0,Oberägeri,2688792.477,1221248.311,de
13726,NaN,NaN,NaN,"Hauptstrasse 44, Oberägeri",12353.0,18540.0,18540.0,frontpage30_3161.htm,1.0,88.0,...,1706.0,Oberägeri,Hauptstrasse,44,6315.0,0.0,Oberägeri,2688768.460,1221217.600,de


In [54]:
#Wie viele Telefonbucheinträge fehlen? Diese habe ich nochmals nachgescrapet - bez. alle die mehr als 5 Firmen hatten.
# Siehe eines weiter unten
df_ht_fe=df_ht1[df_ht1["Anzahl_x"].notnull()==False].sort_values("Anzahl_y", ascending=False)
df_ht_fe.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_fehlende_Telbuch")
df_ht_fe

,Unnamed: 0_x,Seite_x,Anzahl_x,Adresse,Unnamed: 0_y,Unnamed: 0_x,Unnamed: 0.1,Seite_y,Anzahl_y,Unnamed: 0_y,...,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
12198,NaN,NaN,NaN,"Rigiweg 21a, Holzhäusern ZG",10764.0,15300.0,15300.0,frontpage23_58.htm,10.0,1342.0,...,1707.0,Risch,Rigiweg,21a,6343.0,4.0,Holzhäusern ZG,2675835.000,1223131.000,de
13227,NaN,NaN,NaN,"St. Wolfgangstrasse 11, Hünenberg",11846.0,16670.0,16670.0,frontpage30_1294.htm,8.0,530.0,...,1703.0,Hünenberg,St. Wolfgangstrasse,11,6331.0,0.0,Hünenberg,2674739.802,1225725.055,de
13150,NaN,NaN,NaN,"Sihlbruggstrasse 144, Baar",11751.0,16339.0,16339.0,frontpage2_967.htm,8.0,2025.0,...,1701.0,Baar,Sihlbruggstrasse,144,6340.0,0.0,Baar,2685861.551,1229926.361,de
12938,NaN,NaN,NaN,"Am Rainbach 12, Baar",11523.0,16110.0,16110.0,frontpage2_760.htm,7.0,1814.0,...,1701.0,Baar,Am Rainbach,12,6340.0,0.0,Baar,2682923.302,1226564.336,de
13151,NaN,NaN,NaN,"Sihlbruggstrasse 107, Baar",11752.0,16340.0,16340.0,frontpage2_968.htm,7.0,2026.0,...,1701.0,Baar,Sihlbruggstrasse,107,6340.0,0.0,Baar,2685051.729,1229740.914,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12889,NaN,NaN,NaN,"Arbachstrasse 58, Baar",11472.0,16059.0,16059.0,frontpage2_714.htm,1.0,1762.0,...,1701.0,Baar,Arbachstrasse,58,6340.0,0.0,Baar,2682699.955,1225652.453,de
12897,NaN,NaN,NaN,"Arbachstrasse 67, Baar",11480.0,16067.0,16067.0,frontpage2_721.htm,1.0,1769.0,...,1701.0,Baar,Arbachstrasse,67,6340.0,0.0,Baar,2682746.719,1225515.269,de
12904,NaN,NaN,NaN,"Baarermattstrasse 20, Baar",11487.0,16074.0,16074.0,frontpage2_728.htm,1.0,1776.0,...,1701.0,Baar,Baarermattstrasse,20,6340.0,0.0,Baar,2682260.122,1225959.730,de
12905,NaN,NaN,NaN,"Lorzendammweg 11, Baar",11488.0,16075.0,16075.0,frontpage2_729.htm,1.0,1777.0,...,1701.0,Baar,Lorzendammweg,11,6340.0,0.0,Baar,2683419.046,1227661.610,de


In [ ]:
# Ich lasse nur noch jene durch das Telefonbuch, welche mehr als 5 Einträge haben. Die anderen sind definitiv nicht relevant
df_ht_fe5=df_ht_fe[df_ht_fe["Anzahl_y"]>5]
df_ht_fe5.to_csv(folderpath+"/BRFrames/Alle_gemeinden_Telbuch_nachscrapen_51")
df_ht_fe5.shape

In [ ]:
df_ht1.to_csv(folderpath+"/BRFrames/Gemeinden_sauber_HR_und_Telbuch")

# 4. Daten anschauen, hübschen,

- Dataframe so gestalten, dass ich alle infos zur Auswertung habe

In [2]:
df_ht1=pd.read_csv(folderpath+"/BRFrames/Gemeinden_sauber_HR_und_Telbuch")

## 4.1 Klein machen und Zahlen in int umwandeln

In [3]:
# Ich brauche nur einen Teil
df_saukl=df_ht1[["Adresse","Anzahl_y","Seite_y","Anzahl_x","Seite_x","STRNAME","GDENAME","GKODE","GKODN"]]
df_saukl

,Adresse,Anzahl_y,Seite_y,Anzahl_x,Seite_x,STRNAME,GDENAME,GKODE,GKODN
0,"Zugerstrasse 74, Baar",164.0,frontpage2_976.htm,12,telfront10000.htm,Zugerstrasse,Baar,2681933.977,1226416.568
1,"Mühlegasse 18, Baar",92.0,frontpage30_262.htm,57,telfront10001.htm,Mühlegasse,Baar,2682425.299,1228159.836
2,"Blegistrasse 11b, Baar",88.0,frontpage2_936.htm,13,telfront10002.htm,Blegistrasse,Baar,2685739.840,1230044.629
3,"Sihlbruggstrasse 105, Baar",80.0,frontpage2_530.htm,39,telfront10003.htm,Sihlbruggstrasse,Baar,2684903.579,1229684.742
4,"Ruessenstrasse 12, Baar",75.0,frontpage2_963.htm,16,telfront10004.htm,Ruessenstrasse,Baar,2685107.750,1229580.996
...,...,...,...,...,...,...,...,...,...
3495,"Hauptstrasse 28, Oberägeri",1.0,frontpage30_3155.htm,NaN,NaN,Hauptstrasse,Oberägeri,2688917.988,1221178.499
3496,"Früebergstrasse 47, Baar",1.0,frontpage30_316.htm,NaN,NaN,Früebergstrasse,Baar,2682162.578,1228387.387
3497,"Hauptstrasse 40, Oberägeri",2.0,frontpage30_3160.htm,NaN,NaN,Hauptstrasse,Oberägeri,2688792.477,1221248.311
3498,"Hauptstrasse 44, Oberägeri",1.0,frontpage30_3161.htm,NaN,NaN,Hauptstrasse,Oberägeri,2688768.460,1221217.600


In [4]:
# Umbenennen der Columns-Namen
df_saukl.rename(columns={"Anzahl_y":"Firmen", "Anzahl_x":"Telbuch"}, inplace=True)

C:\Users\Zoe\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [5]:
# STADT HATTE EINEN KOMISCHEN EINTRAG
df_saukl1=df_saukl[df_saukl["Telbuch"].str.contains("291'604")==False]# Es hat noch ein komischs Ergebnis
#An der Adresse gibt es aber gar keine Firma. Isch schmeisse diese dehalb gleich weg

In [6]:
# GEMEINDEN HAT AUCH mehrere KOMISCHE
df_saukl1=df_saukl[df_saukl["Telbuch"].str.contains("858'811")==False] 
df_saukl1.sort_values("Telbuch", ascending=False)

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN
41,"Blegistrasse 19, Baar",7.0,frontpage2_938.htm,99,telfront10042.htm,Blegistrasse,Baar,2685545.868,1229989.412
1275,"Sonnenweg 36, Baar",1.0,frontpage0_89.htm,9,telfront8169.htm,Sonnenweg,Baar,2682813.386,1227534.153
340,"Landhausstrasse 21, Baar",6.0,frontpage0_333.htm,9,telfront3082.htm,Landhausstrasse,Baar,2681800.619,1227733.051
342,"Landhausstrasse 21, Baar",6.0,frontpage0_333.htm,9,telfront7082.htm,Landhausstrasse,Baar,2681800.619,1227733.051
821,"Blegistrasse 25, Baar",58.0,frontpage3_615.htm,9,telfront4259.htm,Blegistrasse,Baar,2685436.931,1229905.584
...,...,...,...,...,...,...,...,...,...
1603,"Verenaweg 17, Buonas",7.0,frontpage14_3315.htm,0,telfront7370.htm,Verenaweg,Risch,2677056.270,1221623.001
1602,"Verenaweg 17, Buonas",7.0,frontpage14_3315.htm,0,telfront6370.htm,Verenaweg,Risch,2677056.270,1221623.001
2406,"Zugerstrasse 90, Walchwil",1.0,frontpage14_7952.htm,0,telfront7995.htm,Zugerstrasse,Walchwil,2680780.173,1218246.836
834,"Rebhalde 10, Baar",1.0,frontpage3_678.htm,0,telfront4272.htm,Rebhalde,Baar,2681918.004,1228812.391


In [7]:
#Es hat noch immer komische Einträge. Ersetze das appostroph durch ein 0 und lösche danach die 1000000-er-Einträge
df_saukl1["Telbuch"]=df_saukl1["Telbuch"].str.replace("'","0")

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# Um bei der anschliessenden Division keinen Fehler zu bekommen muss ich noch alle 0-er Telbucheinträge zu 1nern machen.
df_saukl1["Telbuch"]=df_saukl1["Telbuch"].str.replace("0","1") 
#Offenbar erwischt es auch jene 0-er bei 10-nern. Ist aber nicht so tragisch
# Es kommt auf einen mehr oder weniger nicht an

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df_saukl1["Firmen"]=df_saukl1["Firmen"].astype("int")


C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_saukl1["Telbuch"]=df_saukl1["Telbuch"].astype("int")

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_saukl2=df_saukl1.drop_duplicates(subset=["Adresse", "Firmen"])

In [12]:
df_saukl2

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN
0,"Zugerstrasse 74, Baar",164,frontpage2_976.htm,12,telfront10000.htm,Zugerstrasse,Baar,2681933.977,1226416.568
1,"Mühlegasse 18, Baar",92,frontpage30_262.htm,57,telfront10001.htm,Mühlegasse,Baar,2682425.299,1228159.836
2,"Blegistrasse 11b, Baar",88,frontpage2_936.htm,13,telfront10002.htm,Blegistrasse,Baar,2685739.840,1230044.629
3,"Sihlbruggstrasse 105, Baar",80,frontpage2_530.htm,39,telfront10003.htm,Sihlbruggstrasse,Baar,2684903.579,1229684.742
4,"Ruessenstrasse 12, Baar",75,frontpage2_963.htm,16,telfront10004.htm,Ruessenstrasse,Baar,2685107.750,1229580.996
...,...,...,...,...,...,...,...,...,...
3103,"Riedstrasse 12, Rotkreuz",1,frontpage22_403.htm,1,telfront8772.htm,Riedstrasse,Risch,2675038.475,1222263.316
3104,"Grundstrasse 4a, Rotkreuz",1,frontpage22_404.htm,1,telfront8773.htm,Grundstrasse,Risch,2675178.867,1222031.839
3105,"Grundstrasse 16, Rotkreuz",10,frontpage22_405.htm,7,telfront8774.htm,Grundstrasse,Risch,2675292.827,1222175.113
3106,"Grundstrasse 18, Rotkreuz",28,frontpage22_406.htm,17,telfront8775.htm,Grundstrasse,Risch,2675294.944,1222179.278


## 4.2 Wertspalte für Auswertung: Firmen geteilt durch Telbuch
Im Zuge der Recherche/Datenerhebung wurde klar, dass einen Referenzgrösse nötig ist. So führten die Luanda Leaks in die Unter Altstadt, wo in einem Gebäude mit normalerweise drei Parteien 40 Firmen registriert waren. 
Im grössten Firmen-Hub sind wohl über 300 Firmen registriert (die 40 würden also nicht auffallen). Bei den 300 sind aber auch rund 30 Telefonbuch-Einträge zu finden. (Hat auch die Stichprobe für 1-2 anderen der grössten gezeigt.) Teilt man also die Zahl der Firmen durch die Telefonbucheinträge erhält man einen Wert, der eher als Referenz dienen könnte. 

In [13]:
df_saukl2["Vergleich"]=df_saukl2["Firmen"]/df_saukl2["Telbuch"]

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df_saukl2[df_saukl2["Telbuch"]>100].sort_values("Telbuch",ascending=False)

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN,Vergleich
3028,"Mühlegasse, Baar",2,frontpage22_3553.htm,242,telfront8697.htm,Mühlegasse,Baar,2682621.00,1228124.00,0.008264
3041,"Knonauerstrasse, Cham",1,frontpage22_3597.htm,158,telfront8710.htm,Knonauerstrasse,Cham,2677290.00,1226898.00,0.006329
3021,"Deinikon, Baar",4,frontpage22_3535.htm,123,telfront8690.htm,Deinikon,Baar,2683272.00,1228723.00,0.032520
3036,"Altgasse, Baar",1,frontpage22_3582.htm,115,telfront8705.htm,Altgasse,Baar,2681749.64,1227125.19,0.008696


In [15]:
df_saukl2["Firmen"]=df_saukl2["Firmen"].astype(int)

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_saukl2

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN,Vergleich
0,"Zugerstrasse 74, Baar",164,frontpage2_976.htm,12,telfront10000.htm,Zugerstrasse,Baar,2681933.977,1226416.568,13.666667
1,"Mühlegasse 18, Baar",92,frontpage30_262.htm,57,telfront10001.htm,Mühlegasse,Baar,2682425.299,1228159.836,1.614035
2,"Blegistrasse 11b, Baar",88,frontpage2_936.htm,13,telfront10002.htm,Blegistrasse,Baar,2685739.840,1230044.629,6.769231
3,"Sihlbruggstrasse 105, Baar",80,frontpage2_530.htm,39,telfront10003.htm,Sihlbruggstrasse,Baar,2684903.579,1229684.742,2.051282
4,"Ruessenstrasse 12, Baar",75,frontpage2_963.htm,16,telfront10004.htm,Ruessenstrasse,Baar,2685107.750,1229580.996,4.687500
...,...,...,...,...,...,...,...,...,...,...
3103,"Riedstrasse 12, Rotkreuz",1,frontpage22_403.htm,1,telfront8772.htm,Riedstrasse,Risch,2675038.475,1222263.316,1.000000
3104,"Grundstrasse 4a, Rotkreuz",1,frontpage22_404.htm,1,telfront8773.htm,Grundstrasse,Risch,2675178.867,1222031.839,1.000000
3105,"Grundstrasse 16, Rotkreuz",10,frontpage22_405.htm,7,telfront8774.htm,Grundstrasse,Risch,2675292.827,1222175.113,1.428571
3106,"Grundstrasse 18, Rotkreuz",28,frontpage22_406.htm,17,telfront8775.htm,Grundstrasse,Risch,2675294.944,1222179.278,1.647059


In [38]:
df_saukl2.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_ganz_mit_Vergleichswert")

## 4.3 Stadt und Gemeinde zusammenfügen
alle Adressen die mindestens eine Firma haben

In [41]:
df_ht_gem=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_ganz_mit_Vergleichswert")
df_ht_sta=pd.read_csv(folderpath+"/BRFrames/Stadt_ganz_mit_Vergleichswert")                      

In [42]:
df_alles=pd.concat([df_ht_gem,df_ht_sta])

In [45]:
df_alles.to_csv(folderpath+"/BRFrames/Kanton_ganz_mit_Vergleichswert")
df_alles.shape

(6980, 11)

# 5. Sortieren  und anschauen
## Siehe Files: 
- **01.1 Stadt Zug Endprodukt Auswertung** 
- **01.2 Auswertung Geopandas.ipynb**

